In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,ConfusionMatrixDisplay

from tensorflow.keras.utils import load_img,img_to_array
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import RandomCrop, RandomFlip, RandomRotation, RandomContrast
from tensorflow.keras.callbacks import EarlyStopping

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


# Train MobileNetV3
To train our neural network we need:
 - Load the data
 - Downscale data and prepare it for input
 - create some neccessary functions to show image, show scores and etc
 - Load MobileNetV3 pretrained model without top
 - Prepare Data augmentation layers
 - Prepare output layers
 - Set neccessary parameters
 - Train the model
 - evaluate the model performance
 - create pipeline 

#### parameters initialization

In [13]:
# We need to set seed for any randomizing that will happen during process
SEED = 44
# choose how much times we want data to be shown
epochs = 40
BATCH_SIZE = 32
# validation frequency so validation wont be run every epoch
VAL_FREQ = 6
OPTIMIZER = 'adam'

# Create Data Loader
We need data loader to make loading batches computationally efficient and avoid memory issues

In [4]:
def load_from_directory(filename):
    data = tf.keras.utils.image_dataset_from_directory(filename,batch_size=32,image_size=(224,224),seed=seed)
    return data

# We split our dataset into train and validation sets
def split_dir_dataset(data):
    # scale down rgb from 255 to 0
    data = data.map(lambda x,y: (x/255,y))
    
    # percent size of train and val sets
    train_size = int(len(data)*.85)
    val_size = int(len(data)*.15)
    
    # take that size images from dataset
    train = data.take(train_size)
    val = data.skip(train_size).take(val_size)
    
    print(f'training shape: {train.as_numpy_iterator().next()[0].shape}')
    return train,val

In [6]:
# path to our dataset
dataset_path = './multiclass'

data = load_from_directory(dataset_path)

train, val = split_dir_dataset(data)

Found 49842 files belonging to 9 classes.
training shape: (32, 224, 224, 3)


# Necessary Functions
Now we have to initialize necessary functions that we will use later


In [7]:
def show_samples_dataset(data):
    class_names = data.class_names
    plt.figure(figsize=(10, 10))
    for images, labels in data.take(1):
      for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
          
def show_results(hist,mode='acc'):
    if mode == 'acc' or mode == 'accuracy':
        mode = 'accuracy'
        val_mode = 'val_accuracy'
    elif mode == 'loss':
        mode = "loss"
        val_mode = "val_loss"
    
    fig = plt.figure()
    plt.plot(hist.history[mode], color='teal', label=mode)
    plt.plot(hist.history[val_mode], color='orange', label=val_mode)
    fig.suptitle(mode, fontsize=20)
    plt.legend(loc="upper left")
    plt.show()
    
def prediction_dir_dataset(model,test):
    y_true = []
    y_pred = []
    for batch in test.as_numpy_iterator(): 
        X, y = batch
        yhat = model.predict(X)
        yhat = np.argmax(yhat,axis=1)
        y_true.append(y)
        y_pred.append(yhat)
    y_true = np.array(y_true)
    y_true = y_true.reshape(-1)
    y_pred = np.array(y_pred)
    y_pred = y_pred.reshape(-1)
    return y_true,y_pred

def scores(y_true,y_pred,show_confusion_matrix=True):
    precision_micro = precision_score(y_true, y_pred, average='micro')
    precision_weighted = precision_score(y_true, y_pred, average='weighted')
    recall_micro = recall_score(y_true, y_pred, average='micro')
    recall_weighted = recall_score(y_true, y_pred, average='weighted')
    f1_micro = f1_score(y_true, y_pred, average='micro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')

    if show_confusion_matrix == True:
        cm = confusion_matrix(y_true, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        plt.show()
    print(f"precision weighted: {'{:.3f}'.format(precision_weighted)}\nprecision micro: {precision_micro} ")
    print(f'recall weighted: {recall_weighted}\nrecall micro: {recall_micro} ')
    print(f'f1 weighted: {"{:.3f}".format(f1_weighted)}\nf1 micro: {f1_micro} ')

# Load the model

In [16]:
base_model = MobileNetV3Large(weights='imagenet',include_top=False)

In [ ]:
# Global spatial avg pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Fully connected layer
x = Dense(1024,activation='relu')(x)

# We have 2 classes either it is suitable for children or not but for now we train for multiclass classification
# which means softmax distribution over 9 classes
predictions = Dense(9,activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional MobileNetV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
# add Early stopping callback to stop model before it overfits the data
callback = EarlyStopping(monitor='val_loss',patience=5,verbose=1)


In [ ]:
# visualize model
model.summary()

In [ ]:
# initialize this so computing is more efficient
AUTOTUNE = tf.data.experimental.AUTOTUNE

train = train.cache()
train = train.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# train the model on the new data for a few epochs
epochs = 10
hist = model.fit(train,
                 epochs = epochs,
                 validation_data=val,
                 callbacks = [callback],
                 validation_freq=VAL_FREQ)

In [14]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from MobileNet V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

NameError: name 'base_model' is not defined

In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [ ]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=OPTIMIZER), loss=tf.losses.SparseCategoricalCrossentropy())

In [ ]:
# we train our model again, this time fine-tuning the convolutional layers
# alongside the top Dense layers
epochs = 50
hist_unfreezed = model.fit(train,
                           epochs = epochs,
                           validation_data=val,
                           callbacks = [callback],
                           validation_freq=VAL_FREQ)

#  https://keras.io/api/applications/#usage-examples-for-image-classification-models

In [14]:

def predict(img_path):
    # img_path = 'dawg.jpg'
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    preds = model.predict(x)
    # decode the results into a list of tuples (class, description, probability)
    # (one such list for each sample in the batch)
    print('Predicted:', decode_predictions(preds, top=3)[0])